In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/factors-affecting-campus-placement/Placement_Data_Full_Class.csv')
data

In [ ]:
data.info()

Salary has 67 Null values for students who have not been placed

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
sns.set(style='whitegrid', palette='muted', font_scale=1.1)

In [ ]:
data_p = data[data['status'] == 'Placed']
data_np = data[data['status'] == 'Not Placed']
data2 = data.copy()
data2['status'] = data2['status'].map({'Placed':1, 'Not Placed': 0}).astype(int)

# **Heat Map**

In [ ]:
plt.figure(figsize=(14,7))
plt.title('Heatmap')
sns.heatmap(data=data2.drop(['salary', 'sl_no'], axis = 1).corr(), annot = True)

1. High correlation between status and (ssc_p, hsc_p, degree_p)
2. Low correlation between status and (etest_p, mba_p)

# Relationg b/w different markers

In [ ]:
sns.pairplot(data, vars=['ssc_p', 'hsc_p', 'degree_p', 'etest_p', 'mba_p'], hue='status', kind='reg')

# Gender

In [ ]:
data2[['gender','status']].groupby(['gender'], as_index=False).mean()

1. 1. Males have higher prob. of getting placed

# Senior Secondary

In [ ]:
plt.figure(figsize=(12,6))
sns.swarmplot(x=data['status'], y=data['ssc_p'], hue=data['ssc_b'])

In [ ]:
data2[['ssc_b','status']].groupby(['ssc_b'], as_index=False).mean()

In [ ]:
data[['salary', 'ssc_b']].groupby('ssc_b', as_index=False).median()

* No major diffrence in centrer board or other board w.r.t placement probability
* On an average, more percentage score in ssc increases chances of placement
* salary for students of both ssc_b boards is almost same
* Students with ssc_p < 50 are not placed
* Students with ssc_p > 80 are always placed

# High School

In [ ]:
plt.figure(figsize=(12,6))
sns.swarmplot(x=data['status'], y=data['hsc_p'], hue=data['hsc_b'])

In [ ]:
data2[['hsc_b','status']].groupby(['hsc_b'], as_index=False).mean()

In [ ]:
data[['salary', 'hsc_b']].groupby('hsc_b', as_index=False).median()

* No major diffrence in centrer board or other board w.r.t placement probability
* On an average, more percentage score in hsc increases chances of placement
* Median salary for students of both hsc_b boards is almost same

In [ ]:
plt.figure(figsize=(12,6))
sns.swarmplot(x=data['status'], y=data['hsc_p'], hue=data['hsc_s'])

In [ ]:
data2[['hsc_s','status']].groupby(['hsc_s'], as_index=False).mean()

In [ ]:
data[['salary', 'hsc_s']].groupby('hsc_s', as_index=False).median()

* No major diffrence in Science and Commerce w.r.t placement probability. For Arts probability is low.
* Median salary for students in Science and Commerce is almost same. For Arts, the salary is lower.
* We should note that, data points for Arts are less. So, inference w.r.t Art might not be accurate.
* Students with hsc_p < 50 are not placed
* Students with hsc_p > 80 are always placed

# Bachelor's Degree

In [ ]:
plt.figure(figsize=(12,6))
sns.swarmplot(x=data['status'], y=data['degree_p'], hue=data['degree_t'])

In [ ]:
data2[['degree_t','status']].groupby(['degree_t'], as_index=False).mean()

In [ ]:
data[['salary', 'degree_t']].groupby('degree_t', as_index=False).median()

* No major diffrence in Comm&Mgmt and Sci&Tech w.r.t placement probability. For Others probability is lower.
* On an average, more percentage score in bachelors degree increases chances of placement
* In terms of salary there is only a slight diffrence with Sci&Tech being the best paymaster and Others lowest.
* Datapoints for Others is less, so inference from its data might not be accurate
* Students with degree_p < 55 are not placed
* Students with degree_p > 80 are always placed

# Work Experiece


In [ ]:
data2[['workex','status']].groupby(['workex'], as_index=False).mean()

In [ ]:
plt.figure(figsize=(12,6))
sns.barplot(x='workex', y='status', data=data2)

In [ ]:
data[['salary', 'workex']].groupby('workex', as_index=False).median()

* People with prior work experience have a very high chance of placement.
* There is quite a big difference in percentage of people with work experience being placed as compared to freshers.
* Median salary for freshers and experinced professionals is almost similar.

# Employability Test Score


In [ ]:
plt.figure(figsize=(12,6))
sns.swarmplot(x=data['status'], y=data['etest_p'])

In [ ]:
plt.figure(figsize=(12,6))
sns.regplot(x=data['salary'], y=data['etest_p'])

* employability test score does not impact placement.
* Salary seems to increase with increase in employability test score.

# MBA

In [ ]:
plt.figure(figsize=(12,6))
sns.swarmplot(x=data['status'], y=data['mba_p'], hue=data['specialisation'])

In [ ]:
data2[['specialisation','status']].groupby(['specialisation'], as_index=False).mean()

In [ ]:
data2[['specialisation','salary']].groupby(['specialisation'], as_index=False).median()


* No conclusive relation between mba_p and placement
* Mkt&Fin professionals have a higher chance in getting a placement than Mkt&Hr. Infact, 79% of Mkt&Fin get placed compared to just 55% of Mkt&HR
* Mkt&Fin have a slighly higher median salary than Mkt&Hr

# Model Building

In [ ]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [ ]:
data = pd.read_csv('/kaggle/input/factors-affecting-campus-placement/Placement_Data_Full_Class.csv', index_col='sl_no')
data.drop('salary', axis=1, inplace=True)
data['status'] = data['status'].map({'Placed':1, 'Not Placed': 0}).astype(int)
data.head()

In [ ]:
x = data.copy()
x.drop('status', axis=1, inplace = True)
y = data['status']

# Splitting into train and test

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.16, random_state = 1)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

# Encoding categorical variables

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cols = ['gender', 'ssc_b', 'hsc_b', 'hsc_s', 'degree_t', 'workex', 'specialisation']
ohc = OneHotEncoder(handle_unknown='ignore', sparse=False)

n_cols_train = pd.DataFrame(ohc.fit_transform(x_train[cols]))
n_cols_test = pd.DataFrame(ohc.fit_transform(x_test[cols]))

n_cols_train.index = x_train.index
n_cols_test.index = x_test.index

n_cols_train.columns = ohc.get_feature_names(cols)
n_cols_test.columns = ohc.get_feature_names(cols)

x_train = pd.concat([x_train, n_cols_train], axis = 1)
x_test = pd.concat([x_test, n_cols_test], axis = 1)

x_train.drop(cols, axis = 1, inplace = True)
x_test.drop(cols, axis = 1, inplace = True)

In [ ]:
x_train.head()

# Feature Scaling

In [ ]:
mean = x_train.mean()
std = x_train.std()

In [ ]:
x_train = (x_train-mean)/std
x_test = (x_test-mean)/std
x_train.head(3)

# Model Selection

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier
import time
from sklearn.feature_selection import RFE, f_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from pprint import pprint
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [ ]:
models = pd.DataFrame(columns=['model', 'score', 'std','Time to Train']) #DataFrame to store scores of all models

options = [GaussianNB(), 
           LogisticRegression(), 
           SVC(), 
           LinearSVC(), 
           DecisionTreeClassifier(), 
           RandomForestClassifier(), 
           KNeighborsClassifier(), 
           SGDClassifier(), 
           XGBClassifier()]   

model_names = ['Naive Bayes', 
               'Logistic Regression', 
               'Support Vector Machine', 
               'Linear SVC', 
               'Decison Tree',
               'Random Forest',
               'KNN', 
               'SGD Classifier',
               'XGBoost']  

for (opt, name) in zip(options, model_names):
    start=time.time()
    model = opt
    model.fit(x_train, y_train)
    
    scores = cross_val_score(model, x_train, y_train, cv = 5, scoring="accuracy")
    end=time.time()
    row = pd.DataFrame([[name, scores.mean(), scores.std(), end-start]], columns=['model', 'score', 'std','Time to Train'])
    models = pd.concat([models, row], ignore_index=True)

models.sort_values(by='score', ascending=False)

Score is highest for random forest

In [ ]:
rf = RandomForestClassifier(random_state = 3, oob_score=True)
rf.fit(x_train, y_train)
print("OOB Score: ", rf.oob_score_)

# Feature Engineering

In [ ]:
model = RandomForestClassifier(random_state = 3)
model.fit(x_train, y_train)

In [ ]:
rfe = RFE(model, n_features_to_select=1, verbose =3)
rfe.fit(x_train,y_train)

imp1 = pd.DataFrame({'feature':x_train.columns, 'rank1':rfe.ranking_})
imp1 = imp1.sort_values(by = 'rank1')
imp1

In [ ]:
imp2= pd.DataFrame({'featur':x_train.columns, 'importance':np.round(model.feature_importances_, 3)})
imp2['rank2'] = imp2['importance'].rank(ascending=False, method='min')
imp2 = imp2.sort_values(by = 'importance', ascending=False)
imp2

We will combine the above two feature importances and sort on the basis or rank.

In [ ]:
# importances['rank']=importances2['rank'].values
# importances=importances.sort_values('rank')
# importances

imp = pd.concat([imp1, imp2], axis=1)
imp['rank'] = imp['rank1'] + imp['rank2']
imp = imp.sort_values(by = 'rank')
imp = imp.drop(['featur', 'importance', 'rank1', 'rank2'], axis=1)
imp

In [ ]:
x_temp = x_train[imp.feature]

Determing optimal number of features

In [ ]:
features = [i for i in range(22)]
results = []

for i in features:
    rf = RandomForestClassifier(n_jobs=-1, random_state=3)
    cols = x_temp.columns[:i+1]
    x_t = x_temp[cols]
    scores = cross_val_score(rf, x_t, y_train, cv = 5, scoring="accuracy")
    results.append(scores.mean())
    print(i, " : ", np.round(scores.mean(),3), np.round(scores.std(),3))

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))

ax.minorticks_on()

# Customize the major grid
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
# Customize the minor grid
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')

sns.lineplot(y = results, x = features)

* Optimal number of features comes out to be 8
* Making dataset using the optimal number of features

In [ ]:
n_f = 8
to_keep = x_temp.columns[:n_f+1]
x_train_fimp = x_train[to_keep]
x_test_fimp = x_test[to_keep]
x_train_fimp.head()

Checking OOB Score for new dataframe

In [ ]:
rf = RandomForestClassifier(random_state=3, oob_score=True)
rf.fit(x_train_fimp, y_train)
rf.oob_score_

* The new dataset performs better than the original dataset.
* Hence new one is kept as the final dataset

In [ ]:
x_train_final = x_train_fimp
x_test_final = x_test_fimp

# Performing hyperparameter tuning
The current parameters are given as follows: 

In [ ]:
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

* n_estimators = number of trees in the foreset
* max_features = max number of features considered for splitting a node
* max_depth = max number of levels in each decision tree
* min_samples_split = min number of data points placed in a node before the node is split
* min_samples_leaf = min number of data points allowed in a leaf node
* bootstrap = method for sampling data points (with or without replacement)

In [ ]:
rfc=RandomForestClassifier(random_state=42)
param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8,'none']
}
pprint(param_grid)

In [ ]:
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
# Fit the random search model
CV_rfc.fit(x_train_final, y_train)

In [ ]:
CV_rfc.best_params_

In [ ]:
rfc1=RandomForestClassifier(random_state=42, max_features='auto', n_estimators= 200, max_depth=8, criterion='entropy')
rfc1.fit(x_train_final, y_train)

In [ ]:
pred=rfc1.predict(x_test_final)
print("Accuracy for Random Forest after Hyperparameter Tuning on test data: ",accuracy_score(y_test,pred))
pred=rf.predict(x_test_final)
print("Accuracy for Random Forest before Hyperparameter Tuning on test data: ",accuracy_score(y_test,pred))